In [5]:
import requests
import json
import uuid
import os 

from confluent_kafka.avro import AvroProducer
from confluent_kafka import avro

url = os.getenv("CRYPTO_API_URL_ASSETS")

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

if response.status_code == 200:
    data = json.loads(response.text)["data"]
else:
    print("no hay conex")

In [6]:
topic = os.getenv("TOPIC")
bootstrap_servers = os.getenv("BOOTSTRAP_SERVERS")
schema_registry = os.getenv("SCHEMA_REGISTRY")
schema_file = os.getenv("SCHEMA_FILE")
key_schema_string = os.getenv("KEY_SCHEMA_STRING")
# key_schema_string = """
# {"type": "string"}
# """

key_schema = avro.loads(key_schema_string)
value_schema = avro.load(schema_file)

'test'

In [8]:
def avro_value_schema_to_python_types(inputs: dict):
    schema = value_schema.to_json()["fields"]

    avro_schema = {}

    for dic in schema:
        avro_schema[dic["name"]] = dic["type"]

    avro_to_python_types = {
        "int": int,
        "string": str,
        "long": float,
        "float": float,
        "dobule": float,
        "bytes": bytes    
    }
    
    avro_value = None

    for key, val in inputs.items():    
        avro_value = avro_schema[key] if not isinstance(avro_schema[key], list) else avro_schema[key][0]        
        inputs[key] = avro_to_python_types[avro_value](inputs[key]) if inputs[key] is not None else None
        
    return inputs
        
data = [avro_value_schema_to_python_types(data_dic) for data_dic in data]
data

[{'id': 'bitcoin',
  'rank': 1,
  'symbol': 'BTC',
  'name': 'Bitcoin',
  'supply': 18963231.0,
  'maxSupply': 21000000.0,
  'marketCapUsd': 757358169325.7595,
  'volumeUsd24Hr': 6620212233.287136,
  'priceUsd': 39938.24519280283,
  'changePercent24Hr': -0.7508464039672565,
  'vwap24Hr': 39999.63645150823,
  'explorer': 'https://blockchain.info/'},
 {'id': 'ethereum',
  'rank': 2,
  'symbol': 'ETH',
  'name': 'Ethereum',
  'supply': 119652189.249,
  'maxSupply': None,
  'marketCapUsd': 326059728809.07825,
  'volumeUsd24Hr': 6517909807.556441,
  'priceUsd': 2725.062791208422,
  'changePercent24Hr': -3.2126727732464775,
  'vwap24Hr': 2758.522378463274,
  'explorer': 'https://etherscan.io/'},
 {'id': 'tether',
  'rank': 3,
  'symbol': 'USDT',
  'name': 'Tether',
  'supply': 79019501685.78094,
  'maxSupply': None,
  'marketCapUsd': 79403306198.17981,
  'volumeUsd24Hr': 19291687473.670258,
  'priceUsd': 1.0048570859624637,
  'changePercent24Hr': 0.2268242011354477,
  'vwap24Hr': 1.001857738

In [11]:
producer_config = {
    "bootstrap.servers": bootstrap_servers,
    "schema.registry.url": schema_registry
}

producer = AvroProducer(producer_config, default_key_schema=key_schema, default_value_schema=value_schema)

try:
    for values in data:
        record_key = str(uuid.uuid4())
        producer.produce(topic=topic, key=record_key, value=values)
except Exception as e:
    print(f"Exception while producing record value - {values} to topic - {topic}: {e}")
else:
    print(f"Successfully producing record value - {values} to topic - {topic}")
    
producer.flush()

Successfully producing record value - {'id': 'xinfin-network', 'rank': 100, 'symbol': 'XDC', 'name': 'XinFin Network', 'supply': 12293293518.55, 'maxSupply': None, 'marketCapUsd': 677513604.3326694, 'volumeUsd24Hr': 4225206.656694049, 'priceUsd': 0.0551124565040551, 'changePercent24Hr': 2.9178435325032552, 'vwap24Hr': 0.0558739116267885, 'explorer': 'https://etherscan.io/token/0x41ab1b6fcbb2fa9dced81acbdec13ea6315f2bf2'} to topic - test


0

In [2]:
import requests
import json
import uuid
import os

from confluent_kafka.avro import AvroProducer
from confluent_kafka import avro

from map_avro_types import avro_value_schema_to_python_types

TOPIC = os.getenv("TOPIC")
BOOTSTRAP_SERVERS = os.getenv("BOOTSTRAP_SERVERS")
SCHEMA_REGISTRY = os.getenv("SCHEMA_REGISTRY")
SCHEMA_FILE = os.getenv("SCHEMA_FILE")
KEY_SCHEMA_STRING = os.getenv("KEY_SCHEMA_STRING")
CRYPTO_API_URL_ASSETS = os.getenv("CRYPTO_API_URL_ASSETS")

payload = {}
headers = {}

key_schema = avro.loads(KEY_SCHEMA_STRING)
value_schema = avro.load(SCHEMA_FILE)

producer_config = {
    "bootstrap.servers": BOOTSTRAP_SERVERS,
    "schema.registry.url": SCHEMA_REGISTRY,
}

producer = AvroProducer(
    producer_config, default_key_schema=key_schema, default_value_schema=value_schema
)


def main():
    response = requests.request(
        "GET", CRYPTO_API_URL_ASSETS, headers=headers, data=payload
    )

    status = response.status_code

    if response.status_code == 200:
        data = json.loads(response.text)["data"]
    else:
        print(f"Request Exception {status}")

    data = [avro_value_schema_to_python_types(data_dic, value_schema) for data_dic in data]

    try:
        for values in data:
            record_key = str(uuid.uuid4())
            producer.produce(topic=TOPIC, key=record_key, value=values)
    except Exception as e:
        print(
            f"Exception while producing record value - {values} to topic - {TOPIC}: {e}"
        )
    else:
        print(f"Successfully producing record values - count: {len(data)} to topic - {TOPIC}")

    producer.flush()
    
main()

Successfully producing record values - count: 100 to topic - test
